In [1]:
import sys
sys.version

'3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56) \n[GCC 7.3.0]'

# 對pyspark進行設定
spark在local端運行

app的名稱為maxtrix_mul

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("matrix_mul")
sc = SparkContext(conf=conf).getOrCreate()

# 讀取資料

In [3]:
def readData(fileName):
    M = list()
    N = list()
    with open(fileName, 'r') as fp:
        data = fp.readlines()
    for i in range(len(data)):
        data[i] = data[i].split(',')
        if data[i][0] == 'M':
            M.append((int(data[i][1]), int(data[i][2]), int(data[i][3])))
        if data[i][0] == 'N':
            N.append((int(data[i][1]), int(data[i][2]), int(data[i][3])))
    return M, N

# 將Data變成M(k, (i, val))與N(k, (j, val))
首先將M, N轉為rdd的格式

M: Mik => M得第ik個元素
N: Nkj => N的第kj個元素

然後將M, N以k 為key進行儲存，未來可以得到每個Rij

In [4]:
M, N = readData('500input.txt')
M = sc.parallelize(M)
N = sc.parallelize(N)


Mik = M.map(lambda s: (s[1], (s[0], s[2])))
Nkj = N.map(lambda s: (s[0], (s[1], s[2])))

In [5]:
# print(Mik.collect())
# print(Nkj.collect())

# 計算個別的R(i, j) 的值
利用join將所有的k形成(k, (Mik,val), (Nkj, val)) pair

再利用map將結果轉換為(i, j):val

In [6]:
Rij = Mik.join(Nkj)
Rij = Rij.map(lambda s: ((s[1][0][0], s[1][1][0]) ,s[1][0][1]*s[1][1][1]))

In [7]:
# Rij.collect()

# 利用redueBykey將所有相同的key的val相加
得到sum(Rij)

再利用sortedByKey對每個pair進行排序

In [8]:
result = Rij.reduceByKey(lambda x, y: x+y)
sorted_result = result.sortByKey()

sorted_result = sorted_result.collect()

# 將結果寫入檔案

In [9]:
with open("Outputfile.txt", 'w') as fp:
    for i in range(len(sorted_result)):
        idx, val = sorted_result[i]
        row = idx[0]
        col = idx[1]
        fp.write(str(row) + ',' + str(col) + ',' + str(val) + '\n')